In [1]:
#load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

<ipython-input-1-863b54eaad45>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
locator = Nominatim(user_agent="measurements")

In [3]:
df = pd.read_csv(r"T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\bike_share_locations.csv")
df.head()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,OriginDestination
0,14193596,19593,2156242,Knight Law Center,44.043053,-123.069570,Origin
1,14193597,19710,2156242,Knight Law Center,44.043103,-123.069593,Origin
2,14193598,19734,2024943,Eugene Train Station,44.055235,-123.092733,Origin
3,14193600,19617,2203210,NaN,44.044918,-123.085150,Origin
4,14193601,19779,2166326,Erb Memorial Union overflow,44.045250,-123.072697,Origin


In [4]:
df["geom"] = df["Latitude"].map(str) + ',' + df['Longitude'].map(str)
df["geom"][0]

'44.0430533333333,-123.06957'

In [5]:
locator = Nominatim(user_agent="measurements", timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.00001)

In [6]:
tqdm_notebook.pandas()

In [7]:
df['address'] = df['geom'].progress_apply(rgeocode)
df.head()

  0%|          | 0/8886 [00:00<?, ?it/s]

,RouteID,BikeID,UserID,Location,Latitude,Longitude,OriginDestination,geom,address
0,14193596,19593,2156242,Knight Law Center,44.043053,-123.069570,Origin,"44.0430533333333,-123.06957","(University of Oregon, Sunset Alley, Eugene, L..."
1,14193597,19710,2156242,Knight Law Center,44.043103,-123.069593,Origin,"44.0431033333333,-123.069593333333","(University of Oregon, Sunset Alley, Eugene, L..."
2,14193598,19734,2024943,Eugene Train Station,44.055235,-123.092733,Origin,"44.055235,-123.092733333333","(Peace Health Rides, Willamette Street, Eugene..."
3,14193600,19617,2203210,NaN,44.044918,-123.085150,Origin,"44.0449183333333,-123.08515","(1351, Ferry Street, College Hill, Eugene, Lan..."
4,14193601,19779,2166326,Erb Memorial Union overflow,44.045250,-123.072697,Origin,"44.04525,-123.072696666667","(University of Oregon, Sunset Alley, Eugene, L..."


In [8]:
df.address[1]

Location(University of Oregon, Sunset Alley, Eugene, Lane County, Oregon, 97403, United States, (44.044551549999994, -123.07173597767621, 0.0))

In [9]:
df.Location.isnull().values[3]

True

In [10]:
def getLoc(x):
    loc = x.raw['address']
    a = [key for key, value in loc.items()][0]
    if  a == 'house_number':
        res = loc['road'] + ' ' + loc['house_number']
    else:
        res = loc[a]
    return res

In [11]:
getLoc(df[df.Location.isnull().values].address.values[1])

'Millrace Studios'

In [12]:
df['Name'] = df.address.apply(lambda x: getLoc(x))

In [13]:
df.loc[df.Location.isnull(), 'Location'] = df.loc[df.Location.isnull(), 'Name']

In [18]:
for i in df.RouteID.unique():
    idx1 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Origin'))
    idx2 = ((df['RouteID'] == i) & (df['OriginDestination'] =='Destination'))
    pathID = df.loc[idx1, 'Location'].values[0] + ' - ' + df.loc[idx2, 'Location'].values[0]
    df.loc[idx1, 'PathID'] = pathID
    df.loc[idx2, 'PathID'] = pathID

In [19]:
df.tail()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,OriginDestination,PathID
8881,14209403,19712,2211448,17th & Hilyard,44.041062,-123.081933,Destination,"17th & Pearl, West side - 17th & Hilyard"
8882,14209412,19763,1733742,Glenwood,44.045285,-123.080093,Destination,Monroe St & Blair Blvd - Glenwood
8883,14209416,19631,1354450,Maude Kerns Art Center,44.042795,-123.064628,Destination,13th & Kincaid - Maude Kerns Art Center
8884,14209417,19755,1354450,Villard Street 1544,44.042743,-123.064662,Destination,13th & Kincaid - Villard Street 1544
8885,14209418,19628,1354450,Villard Street 1532,44.042865,-123.064647,Destination,13th & Kincaid - Villard Street 1532


In [16]:
ndf = df.drop(['geom', 'address', 'Name'], axis=1)

In [20]:
ndf.to_csv(r'T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\pathID.csv', index=False)